In [ ]:
# Step 1: Import Libraries
import os
from dotenv import load_dotenv
from langchain_community.llms import HuggingFaceHub
import json, getpass, pathlib
import pathlib, subprocess, zipfile
import pandas as pd
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
from tqdm.auto import tqdm
import pickle
import textwrap, html
from IPython.display import display, Markdown
from functools import lru_cache
import yaml
from huggingface_hub import InferenceClient
from IPython.display import display, Markdown
import textwrap

In [ ]:
# Step 2: Load Hugging Face API Token

load_dotenv()

token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not token:
    # fallback: prompt user inside notebook
    token = input("Enter your Hugging Face API token: ").strip()
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = token

print("✅ Hugging Face API key loaded")


In [ ]:
# Step 3:  Setup Hugging Face LLM (no fallback)
from langchain.llms import HuggingFaceHub

# Example model: Falcon-7B-Instruct (fast + free-tier friendly)
try:
    llm = HuggingFaceHub(
        repo_id=os.getenv("HF_LLM_MODEL", "tiiuae/falcon-7b-instruct"),
        model_kwargs={"temperature":0.6, "max_new_tokens":512}
    )
    print("✅ Hugging Face LLM loaded")
except Exception as e:
    print(f"⚠️ Failed to load LLM: {e}")

In [ ]:
# Step 4: One-time Kaggle API credentials (improved)
import json, getpass, pathlib

kaggle_dir = pathlib.Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)
kaggle_json = kaggle_dir / "kaggle.json"

if not kaggle_json.exists():
    username = os.getenv("KAGGLE_USERNAME")
    key = os.getenv("KAGGLE_KEY")
    if not username or not key:
        print("Enter Kaggle credentials (leave blank to skip if already configured):")
        username = input("Kaggle Username: ").strip()
        key = getpass.getpass("Kaggle API Key: ").strip()
    if username and key:
        kaggle_json.write_text(json.dumps({"username": username, "key": key}))
        os.chmod(kaggle_json, 0o600)
        print("✅ Saved ~/.kaggle/kaggle.json")
    else:
        print("ℹ️ Skipped writing kaggle.json (assuming already configured).")
else:
    print("✅ Found existing ~/.kaggle/kaggle.json")


In [ ]:
# Step 5: Download & unzip Kaggle dataset (Stack Overflow: stacksample)
import pathlib, subprocess, zipfile

DATA_DIR = pathlib.Path("./data")
DATA_DIR.mkdir(exist_ok=True, parents=True)

zip_path = DATA_DIR / "stacksample.zip"
if not zip_path.exists():
    subprocess.run(["kaggle", "datasets", "download", "-d", "stackoverflow/stacksample", "-p", "./data", "-q"])

if zip_path.exists():
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(DATA_DIR)
    print("✅ Extracted stacksample")
else:
    print("⚠️ Could not find the downloaded zip. Ensure Kaggle config is correct.")

print("Files:", [p.name for p in DATA_DIR.iterdir()])

In [ ]:
# Step 6: Load Stack Overflow sample data; prepare a small tag-index for retrieval
import pandas as pd
import numpy as np
from pathlib import Path

q_path = Path("./data/Questions.csv")
a_path = Path("./data/Answers.csv")
t_path = Path("./data/Tags.csv")

questions = pd.read_csv(q_path, encoding="latin-1")
answers = pd.read_csv(a_path, encoding="latin-1")
tags = pd.read_csv(t_path, encoding="latin-1")

questions = questions.rename(columns={"Id":"QuestionId","OwnerUserId":"UserId"})
answers = answers.rename(columns={"Id":"AnswerId","OwnerUserId":"UserId"})

tag_map = tags.groupby("Id")["Tag"].apply(list).rename("TagList")
questions = questions.merge(tag_map, left_on="QuestionId", right_index=True, how="left")

ARCH_TAGS = {
    "architecture","design","microservices","monolith","rest","graphql","event-driven",
    "serverless","kubernetes","docker","kafka","rabbitmq","cassandra","redis",
    "mongodb","postgresql","mysql","elasticsearch","aws","gcp","azure","scalability",
    "performance","latency","availability","resilience","cqrs","ddd","clean-architecture",
    "hexagonal-architecture","pubsub","load-balancing","caching","db-sharding","rate-limiting"
}

def has_arch_tag(taglist):
    if isinstance(taglist, list):
        cleaned = [str(t).lower() for t in taglist if isinstance(t, str)]
        return bool(set(cleaned) & ARCH_TAGS)
    return False

arch_q = questions[questions["TagList"].apply(has_arch_tag)].copy()
arch_q["Text"] = (arch_q["Title"].fillna("") + " " + arch_q["Body"].fillna("")).str.replace(r"<[^>]+>", " ", regex=True)

arch_q = arch_q[["QuestionId","Score","Title","Text","TagList"]].reset_index(drop=True)

print("Total questions:", len(questions))
print("Architecture-like questions:", len(arch_q))
arch_q.head(3)

In [ ]:
# Step 7: Build embeddings and FAISS index
from sentence_transformers import SentenceTransformer
import faiss
from tqdm.auto import tqdm
import pickle

EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("Loading embedding model:", EMB_MODEL_NAME)
embed_model = SentenceTransformer(EMB_MODEL_NAME)

def chunk_text(text, max_tokens=250, sep="\n"):
    if not isinstance(text, str):
        return []
    parts = text.split(sep)
    out, cur, cur_len = [], [], 0
    for p in parts:
        plen = len(p.split())
        if cur_len + plen <= max_tokens:
            cur.append(p)
            cur_len += plen
        else:
            if cur:
                out.append(sep.join(cur))
            cur = [p]
            cur_len = plen
    if cur:
        out.append(sep.join(cur))
    final = []
    for piece in out:
        words = piece.split()
        if len(words) <= max_tokens:
            final.append(piece)
        else:
            for i in range(0, len(words), max_tokens):
                final.append(" ".join(words[i:i+max_tokens]))
    return final

texts, metadatas = [], []
for idx, row in tqdm(arch_q.iterrows(), total=len(arch_q), desc="Preparing chunks"):
    chunks = chunk_text(row["Text"], max_tokens=200)
    if not chunks:
        continue
    for i, c in enumerate(chunks):
        texts.append(c)
        metadatas.append({
            "QuestionId": int(row["QuestionId"]),
            "Title": row["Title"][:240],
            "TagList": row["TagList"],
            "Score": int(row["Score"]) if not pd.isna(row["Score"]) else None,
            "chunk_id": i
        })

print(f"Prepared {len(texts)} text chunks from {len(arch_q)} questions.")

BATCH = 256
embeddings = []
for i in tqdm(range(0, len(texts), BATCH), desc="Embedding batches"):
    batch_texts = texts[i:i+BATCH]
    emb = embed_model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
    embeddings.append(emb)
embeddings = np.vstack(embeddings).astype("float32")
print("Embeddings shape:", embeddings.shape)

faiss.normalize_L2(embeddings)
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embeddings)
print("FAISS index built. Total vectors:", index.ntotal)

os.makedirs("kb", exist_ok=True)
faiss.write_index(index, "kb/arch_index.faiss")
with open("kb/arch_texts.pkl", "wb") as f:
    pickle.dump(texts, f)
with open("kb/arch_metas.pkl", "wb") as f:
    pickle.dump(metadatas, f)

print("Saved index and metadata under ./kb/")


In [ ]:
# Step 8: Retrieval helper functions
import textwrap, html
from IPython.display import display, Markdown
from functools import lru_cache

if 'index' not in globals():
    index = faiss.read_index("kb/arch_index.faiss")
if 'texts' not in globals():
    with open("kb/arch_texts.pkl","rb") as f:
        texts = pickle.load(f)
if 'metadatas' not in globals():
    with open("kb/arch_metas.pkl","rb") as f:
        metadatas = pickle.load(f)

@lru_cache(maxsize=128)
def retrieve_knn(query, k=5, return_scores=False):
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    results = []
    for dist, idx in zip(D[0], I[0]):
        if idx < 0 or idx >= len(texts):
            continue
        meta = metadatas[idx]
        snippet = texts[idx]
        item = {
            "score": float(dist),
            "qid": meta.get("QuestionId"),
            "title": meta.get("Title"),
            "tags": meta.get("TagList"),
            "chunk_id": meta.get("chunk_id"),
            "snippet": snippet
        }
        results.append(item if return_scores else {k:v for k,v in item.items() if k!="score"})
    return results

def show_retrieval(results):
    out = []
    for i, r in enumerate(results, 1):
        s = textwrap.shorten(r["snippet"], width=350, placeholder="…")
        tags = ", ".join(r["tags"]) if r["tags"] else ""
        out.append(f"**[{i}]** — *QID {r['qid']}* — tags: {tags}\n\n> {s}\n")
    display(Markdown("\n\n".join(out)))

res = retrieve_knn("design event-driven microservices with kafka, idempotency, retries", k=4)
show_retrieval(res)

In [ ]:
# Step 9: Patterns KB (moved to external YAML recommended, but kept inline here)
import yaml

PATTERNS_YAML = """
patterns:
  - name: Monolith
    description: "Single deployable unit; simple ops; quick to iterate."
    use_when:
      - "Small team"
      - "Tight timelines"
    tech_stack:
      conservative: ["Python", "FastAPI", "PostgreSQL", "Redis"]
      balanced: ["Node.js", "Express", "Postgres", "Redis"]
      innovative: ["Deno", "PlanetScale", "Upstash Redis"]
  - name: Microservices
    description: "Multiple services per bounded context; independent deploys."
    use_when:
      - "Large teams"
      - "Need independent scaling"
    tech_stack:
      conservative: ["Java", "Spring Boot", "Postgres", "Kafka", "Kubernetes"]
      balanced: ["Go", "gRPC", "Kafka", "MongoDB", "Kubernetes"]
      innovative: ["Rust", "NATS", "CockroachDB", "Service Mesh"]
  - name: Event-Driven
    description: "Asynchronous comms using event bus; good for decoupling."
    use_when:
      - "High throughput"
      - "Loose coupling required"
    tech_stack:
      conservative: ["Kafka", "Schema Registry", "Postgres"]
      balanced: ["RabbitMQ", "Consumers in Python/Go", "Elastic sink"]
      innovative: ["Redpanda", "Flink", "Delta Lake"]
  - name: Serverless
    description: "Fully managed function-based architecture."
    use_when:
      - "Spiky workloads"
      - "Minimal ops"
    tech_stack:
      conservative: ["AWS Lambda", "API Gateway", "DynamoDB"]
      balanced: ["GCP Cloud Run + Functions", "Firestore"]
      innovative: ["Dapr", "Cloud-native serverless stacks"]
"""

PATTERNS = yaml.safe_load(PATTERNS_YAML)["patterns"]

def find_pattern(name):
    name = name.lower()
    for p in PATTERNS:
        if name in p["name"].lower():
            return p
    return None

def list_patterns():
    return [p["name"] for p in PATTERNS]

print("Available patterns:", list_patterns())

In [ ]:
# Step 10: Initialize InferenceClient (reuse token)
from huggingface_hub import InferenceClient

hf_client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)

In [ ]:
# Step 11: Setup Hugging Face Client and Define Functions for Cleaning & Generating Proposals

import re
from IPython.display import Markdown, display

MODEL_ID = "HuggingFaceH4/zephyr-7b-beta"
hf_client = InferenceClient(model=MODEL_ID, token=os.environ["HUGGINGFACEHUB_API_TOKEN"])

def _clean_llm_text(text: str) -> str:
    """Clean and de-duplicate LLM output, collapse repeated adjacent phrases,
    limit bullets per section, and produce tidy Markdown."""
    if not text:
        return ""

    # 1) Remove common unwanted markers and control tokens
    text = re.sub(r"\[/?ASSIST(?:ANT)?\]", "", text, flags=re.IGNORECASE)
    text = re.sub(r"<\/?s>", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\[/?USER\]", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\[/?SYSTEM\]", "", text, flags=re.IGNORECASE)

    # 2) Normalize whitespace and line endings
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)  # no huge gaps

    # 3) Break into lines, strip, remove empty lines
    lines = [ln.strip() for ln in text.split("\n")]
    lines = [ln for ln in lines if ln != ""]

    # 4) Collapse long repeated adjacent lines (e.g., the same operator repeated)
    collapsed = []
    prev = None
    for ln in lines:
        if ln == prev:
            # skip exact adjacent duplicate
            continue
        collapsed.append(ln)
        prev = ln
    lines = collapsed

    # 5) Deduplicate while preserving order (but keep short duplicates within same section)
    seen = set()
    deduped = []
    for ln in lines:
        key = ln.lower()
        # For long lines we keep if new; for tiny repeated tokens also dedupe
        if key in seen:
            continue
        seen.add(key)
        deduped.append(ln)
    lines = deduped

    # 6) Re-scan for headings and group sections, truncating bullet lists per section
    # Recognize headings like "1.", "1)", "1 -", or Markdown bold headings (**Recommended Pattern**)
    sections = []
    cur_title = "Intro"
    cur_lines = []
    heading_re = re.compile(r"^\s*(\d+[\.\)\-]|#+\s+|\*\*.+\*\*|[A-Za-z ]+:)\s*")
    for ln in lines:
        if heading_re.match(ln):
            # flush previous
            if cur_lines:
                sections.append((cur_title, cur_lines))
            # treat the heading as the new title (clean it)
            title = ln
            title = re.sub(r"^\d+[\.\)\-]\s*", "", title)  # remove leading number
            title = title.strip()
            cur_title = title if title else "Section"
            cur_lines = []
        else:
            cur_lines.append(ln)
    if cur_lines:
        sections.append((cur_title, cur_lines))

    # 7) For each section, keep at most N bullets and collapse long repetitive sentences into bullets
    MAX_BULLETS = 6
    out_sections = []
    for title, s_lines in sections:
        # Convert lines into bullets: split by common bullet markers if present
        bullets = []
        for ln in s_lines:
            # if line starts with a dash / bullet, strip it
            if re.match(r"^[-•\*\u2022]\s+", ln):
                ln = re.sub(r"^[-•\*\u2022]\s+", "", ln)
            # Split comma-heavy sentences into separate bullets only if clearly enumerative and short
            if len(ln) > 200:
                # long paragraph — keep as single bullet but shorten
                ln = re.sub(r"\s+", " ", ln)  # collapse internal whitespace
            bullets.append(ln)

        # dedupe bullets while preserving order
        seen_b = set()
        uniq_bullets = []
        for b in bullets:
            key = b.lower()
            if key in seen_b:
                continue
            seen_b.add(key)
            uniq_bullets.append(b)

        # truncate
        uniq_bullets = uniq_bullets[:MAX_BULLETS]

        # If no explicit bullets, try splitting long lines into sensible bullets using sentences.
        if len(uniq_bullets) == 1 and len(uniq_bullets[0]) > 300:
            spl = re.split(r"(?<=\.)\s+", uniq_bullets[0])
            uniq_bullets = [s.strip() for s in spl if s.strip()][:MAX_BULLETS]

        out_sections.append((title, uniq_bullets))

    # 8) Reconstruct Markdown output with titles and bullets
    md_parts = []
    for title, bullets in out_sections:
        if title.lower() != "intro":
            md_parts.append(f"### {title.strip()}")
        for b in bullets:
            # ensure bullet starts with "- "
            md_parts.append(f"- {b}")
        md_parts.append("")  # blank line between sections

    out_md = "\n".join(md_parts).strip()
    # Final cleanup: avoid duplicated blank lines
    out_md = re.sub(r"\n{3,}", "\n\n", out_md)
    return out_md

def generate_proposals(requirement: str, k: int = 3) -> str:
    # retrieve and assemble context
    retrieved = retrieve_knn(requirement, k=k, return_scores=True)
    context = "\n\n".join([f"- {r['snippet']}" for r in retrieved])

    # stricter system prompt that tells the model to be concise and limited
    system_prompt = """
You are an expert cloud architect.
Always respond in a concise Markdown format with headings and bullets.
Keep each section short (max ~6 bullets). Never repeat the same advice.
If the user asks for optimizations, provide at most 5 practical, non-overlapping tips.
Do not enumerate low-level operators or repeat identical commands.
""".strip()

    user_prompt = (
        f"Requirement: {requirement}\n\n"
        f"Context snippets:\n{context}\n\n"
        "Provide a structured recommendation using headings and bullets."
    )

    response = hf_client.chat_completion(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1024,
        temperature=0.3,
        top_p=0.9
    )

    raw = response.choices[0].message.get("content", "")
    cleaned = _clean_llm_text(raw)
    # If cleaning produced nothing, fall back to raw (cleaned minimally)
    return cleaned if cleaned else raw


In [ ]:
# Step12: Interactive Chat Assistant Loop (Markdown display)
from IPython.display import Markdown, display
import textwrap

chat_history = []

def chat_with_architect():
    global chat_history
    print("🤖 Conversational Code Architecture Assistant (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("👋 Ending session.")
            break

        try:
            rec_md = generate_proposals(user_input, k=3)
            chat_history.append({"role": "user", "content": user_input})
            chat_history.append({"role": "assistant", "content": rec_md})

            # Render as Markdown for clean headings & bullets
            display(Markdown(rec_md))
            print("\n" + "-"*100 + "\n")
        except Exception as e:
            print(f"⚠️ Error generating response: {e}")
            break

# Start the chat
chat_with_architect()
